In [1]:
%config Completer.use_jedi = False

In [2]:
import os
import flask
import numpy as np
import time
from PIL import Image
from scipy.linalg import sqrtm

from tqdm import tqdm

import torch

from sklearn.cluster import KMeans

from flask import Flask
from flask_cors import CORS
import math

from matplotlib import pyplot as plt

import umap

In [223]:
# create Flask app
app = Flask(__name__)
app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 0
CORS(app)

# TODO load all of the data generated from preprocessing
qt = torch.load('./static/qt.pt')
tensor2_3 = torch.load('./static/tensor2_3.pt')
tensor2_3 = torch.nan_to_num(tensor2_3,nan=0.0) # Ajustando casos nan

tensor3_4 = torch.load('./static/tensor3_4.pt')
tensor3_4 = torch.nan_to_num(tensor3_4,nan=0.0) #  Ajustando casos nan

images = []
for i in range(0,20):
    images.append(Image.open('./static/image'+str(i)+'.jpeg'))

# number of clusters - feel free to adjust
n_clusters = 9

# these variables will contain the clustering of channels for the different layers
a2_clustering,a3_clustering,a4_clustering = None,None,None

In [349]:
A =[[0.,1.,1.,0.,0.,1.,0.,0.,1.,1.],
    [1.,0.,1.,0.,0.,0.,0.,0.,0.,0.],
    [1.,1.,0.,0.,0.,0.,0.,0.,0.,0.],
    [0.,0.,0.,0.,1.,1.,0.,0.,0.,0.],
    [0.,0.,0.,1.,0.,1.,0.,0.,0.,0.],
    [1.,0.,0.,1.,1.,0.,1.,1.,0.,0.],
    [0.,0.,0.,0.,0.,1.,0.,1.,0.,0.],
    [0.,0.,0.,0.,0.,1.,1.,0.,0.,0.],
    [1.,0.,0.,0.,0.,0.,0.,0.,0.,1.],
    [1.,0.,0.,0.,0.,0.,0.,0.,1.,0.]]
A = np.array(A)

In [366]:
eg = np.linalg.eig(A)[0]

array([-2.23606798, -1.        , -1.        , -1.        , -1.        ,
       -1.        ,  1.        ,  1.        ,  2.23606798,  3.        ])

In [360]:
At=torch.FloatTensor(A)
vals, vecs = torch.eig(At,eigenvectors=True)

In [203]:
from sklearn.cluster import SpectralClustering
# spectral_clustering(A,4)
spec = SpectralClustering(n_clusters=4,affinity="precomputed")
spec.fit(A)
spec.labels_

array([2, 2, 2, 3, 3, 0, 0, 0, 1, 1], dtype=int32)

In [383]:
vals[torch.argsort(vals[:,0],descending=True),0]

tensor([ 3.0000,  2.2361,  1.0000,  1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -2.2361])

In [385]:
vecs.shape[]

torch.Size([10, 10])

In [467]:
row_norms = torch.norm(vecs[:,0:2],dim=1).unsqueeze(1).repeat(1,2)

vecs[:,0:2]/row_norms

tensor([[-0.7136, -0.7006],
        [-0.7839,  0.6209],
        [-0.5493,  0.8356],
        ...,
        [-0.6636, -0.7481],
        [-0.9717, -0.2362],
        [-0.8714,  0.4905]])

In [475]:
'''
TODO

Implement spectral clustering, given an affinity matrix. You are required to implement this using standard matrix computation libraries, e.g. numpy, for computing a spectral embedding.
You may use k-means once you've obtained the spectral embedding.

NOTE: the affinity matrix should _not_ be symmetric! Nevertheless, eigenvectors will be real, up to numerical precision - so you should cast to real numbers (e.g. np.real).
'''
def spectral_clustering(affinity_mat, n_clusters):
    # code based on https://towardsdatascience.com/spectral-clustering-aba2640c0d5b
    # diagonal matrix
    D = torch.diag(torch.sum(affinity_mat,dim=1))

    # graph laplacian
    L = torch.pow(torch.inverse(D),1/2) @ affinity_mat @ torch.pow(D,1/2)

    # eigenvalues and eigenvectors
    vals, vecs = torch.eig(L,eigenvectors=True)
    vals = vals[:,0]
    # sort these based on the eigenvalues
    vals = vals[torch.argsort(vals,descending=True)]
    vecs = vecs[:,torch.argsort(vals,descending=True)]
    row_norms = torch.norm(vecs[:,0:2],dim=1).unsqueeze(1).repeat(1,2)

    Y = vecs[:,0:2]/row_norms
    
    # kmeans on first three vectors with nonzero eigenvalues
    kmeans = KMeans(n_clusters=n_clusters,random_state=7)
    kmeans.fit(Y[:,1:n_clusters])
    labels = kmeans.labels_
    return labels

In [478]:
spectral_clustering(s2,9)

array([0, 7, 3, 6, 3, 0, 0, 4, 4, 5, 6, 1, 7, 7, 0, 0, 3, 2, 0, 0, 0, 4,
       3, 0, 0, 3, 4, 0, 5, 3, 8, 6, 0, 1, 0, 4, 6, 8, 4, 6, 0, 7, 8, 7,
       7, 4, 3, 5, 8, 2, 0, 2, 0, 6, 0, 5, 2, 5, 6, 5, 0, 3, 3, 3, 3, 1,
       8, 7, 1, 3, 6, 0, 3, 7, 4, 6, 6, 4, 2, 0, 6, 8, 2, 5, 0, 7, 6, 0,
       4, 0, 3, 0, 3, 6, 0, 3, 3, 1, 0, 3, 6, 0, 7, 0, 4, 0, 8, 0, 2, 3,
       3, 5, 4, 6, 6, 6, 5, 4, 0, 0, 4, 1, 5, 0, 0, 1, 3, 7, 4, 0, 4, 8,
       5, 2, 3, 6, 6, 3, 8, 4, 0, 1, 6, 7, 7, 6, 1, 0, 6, 4, 2, 7, 7, 2,
       6, 5, 3, 0, 4, 6, 3, 5, 4, 6, 4, 8, 3, 3, 5, 0, 2, 4, 1, 8, 5, 4,
       3, 0, 2, 3, 7, 6, 3, 1, 3, 3, 7, 3, 3, 1, 0, 8, 2, 6, 3, 0, 6, 7,
       1, 1, 3, 0, 3, 1, 4, 5, 0, 3, 0, 3, 6, 0, 0, 8, 5, 0, 0, 6, 6, 4,
       7, 3, 8, 2, 1, 0, 8, 5, 6, 4, 4, 6, 2, 3, 1, 5, 2, 4, 1, 5, 2, 0,
       6, 2, 7, 1, 6, 4, 2, 1, 1, 0, 3, 3, 4, 1, 6, 3, 1, 3, 1, 7, 5, 3,
       6, 2, 0, 4, 4, 7, 1, 5, 3, 4, 1, 8, 6, 3, 3, 6, 3, 5, 3, 4, 8, 7,
       5, 2, 0, 5, 6, 3, 5, 0, 3, 3, 7, 3, 2, 2, 7,

tensor([[-0.0442, -0.0434],
        [-0.0442,  0.0350],
        [-0.0442,  0.0672],
        ...,
        [-0.0442, -0.0498],
        [-0.0442, -0.0107],
        [-0.0442,  0.0249]])

In [312]:
v = torch.eig(r23)

In [329]:
D = torch.diag(torch.sum(s2,dim=1))
L = torch.pow(torch.inverse(D),1/2) @ s2 @ torch.pow(D,1/2)

In [346]:
L = torch.pow(torch.inverse(D),1/2) @ s2 @ torch.pow(D,1/2)
A = torch.FloatTensor(A)

In [ ]:
np

In [348]:
torch.eig(A)

torch.return_types.eig(
eigenvalues=tensor([[-2.2361,  0.0000],
        [ 3.0000,  0.0000],
        [ 2.2361,  0.0000],
        [-1.0000,  0.0000],
        [-1.0000,  0.0000],
        [ 1.0000,  0.0000],
        [-1.0000,  0.0000],
        [ 1.0000,  0.0000],
        [-1.0000,  0.0000],
        [-1.0000,  0.0000]]),
eigenvectors=tensor([]))

In [284]:
s23 = tensor2_3.sum(dim=0)
s34 = tensor2_3.sum(dim=0)

c23 = torch.diag(s23.sum(dim=0))
r23 = torch.diag(s23.sum(dim=1))

c34 = torch.diag(s34.sum(dim=0))
r34 = torch.diag(s34.sum(dim=1))

s2  = torch.inverse(r23) @ s23 @ torch.transpose(s23 @ torch.inverse(c23),0,1)
s4  = torch.transpose(s34 @ torch.inverse(c34),0,1) @ torch.inverse(r34) @ s34 
s3  = torch.transpose(s23 @ torch.inverse(c23),0,1) @ torch.inverse(r23) @ s23 \
        + torch.inverse(r34) @ s34 @ torch.transpose(s34 @ torch.inverse(c34),0,1)

In [321]:
torch.pow(torch.inverse(r34),1/2) - sqrtm(R)
R = r34.numpy()
# R = np.linalg.inv(R)
# sqrtm(R)

In [32]:
'''
Do not cache images on browser, see: https://stackoverflow.com/questions/34066804/disabling-caching-in-flask
'''
@app.after_request
def add_header(r):
    r.headers['Cache-Control'] = 'no-cache, no-store, must-revalidate'
    r.headers['Pragma'] = 'no-cache'
    r.headers['Expires'] = '0'
    r.headers['Cache-Control'] = 'public, max-age=0'
    return r
#

    

'''
TODO

Cluster the channels within each layer.
This should take, as arguments, the two similarity matrices derived from the IoU scores.
Specifically, the first argument is the similarity matrix between channels at layer 2 and channels at layer 3.
The second argument is the similarity matrix between channels at layer 3 and channels at layer 4.

A generalization of spectral biclustering should be performed. More details given in the assignment notebook.
'''
def multiway_spectral_clustering(sim_a2_a3, sim_a3_a4, n_clusters):
    pass
#

'''
TODO

Given a link selected from the visualization, namely the layer and clusters at the individual layers, this route should compute the mean correlation from all channels in the source layer and all channels in the target layer, for each sample.
'''
@app.route('/link_score', methods=['GET','POST'])
def link_score():
    pass
#

'''
TODO

Given a layer (of your choosing), perform max-pooling over space, giving a vector of activations over channels for each sample. Perform UMAP to compute a 2D projection.
'''
@app.route('/channel_dr', methods=['GET','POST'])
def channel_dr():
    pass
#

'''
TODO

Compute correlation strength over selected instances, those brushed by the user.
'''
@app.route('/selected_correlation', methods=['GET','POST'])
def selected_correlation():
    pass
#

'''
TODO

Compute correlation strength over all instances.
'''
@app.route('/activation_correlation_clustering', methods=['GET'])
def activation_correlation_clustering():
    pass
#

'''
TODO

In the main, before running the server, run clustering, store results in variables a2_clustering, a3_clustering, a4_clustering
'''
# if __name__=='__main__':
#     app.run()
#

'\nTODO\n\nCluster the channels within each layer.\nThis should take, as arguments, the two similarity matrices derived from the IoU scores.\nSpecifically, the first argument is the similarity matrix between channels at layer 2 and channels at layer 3.\nThe second argument is the similarity matrix between channels at layer 3 and channels at layer 4.\n\nA generalization of spectral biclustering should be performed. More details given in the assignment notebook.\n'